In [24]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import torch
import torch.nn as nn
from tqdm import tqdm

In [43]:
df = pd.read_csv('../claw-data-Mar22.csv', header=None)
df.columns = ["timestamp", "s0", "s1", "s2", "label"]

In [44]:
df.head()

,timestamp,s0,s1,s2,label
0,776864599.0,1.529956,1.483227,1.497729,0.0
1,776869620.0,1.519482,1.490479,1.509814,0.0
2,776874637.0,1.508203,1.490479,1.504980,0.0
3,776879654.0,1.535596,1.496924,1.495312,0.0
4,777266082.0,1.533179,1.488867,1.513037,0.0


In [45]:
len(df)

382

In [46]:
df[['s0', 's1', 's2']][1:1+5].to_numpy()

array([[1.519482, 1.490479, 1.509814],
       [1.508203, 1.490479, 1.50498 ],
       [1.535596, 1.496924, 1.495312],
       [1.533179, 1.488867, 1.513037],
       [1.51062 , 1.492895, 1.496118]])

In [182]:
data = []
labels = []

window = 20

for i in range(len(df)-window):
    data.append(df[['s0', 's1', 's2']][i:i+window].to_numpy().reshape(-1))
    labels.append(df['label'][i:i+1].to_numpy())
    
data = np.array(data)
labels = np.array(labels).ravel()

In [183]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)

In [184]:
train_labels.shape

(289,)

In [185]:
logistic = LogisticRegression(random_state=1)
# logistic = logistic.fit(train_data, train_labels)

In [186]:
scores = cross_val_score(logistic, data, labels, cv=5)
scores

array([0.5890411 , 0.57534247, 0.58333333, 0.59722222, 0.58333333])

In [187]:
X, y = [], []
for i in range(len(df)-window):
    feature = df[['s0', 's1', 's2']][i:i+window].to_numpy()/3.3
    target = df['label'][i+1:i+window+1].to_numpy()
    X.append(feature)
    y.append(target)
X, y = torch.tensor(np.array(X).astype(np.float32)), torch.tensor(np.array(y).astype(np.float32))
X, y = X, y

In [188]:
X.shape, y.shape

(torch.Size([362, 20, 3]), torch.Size([362, 20]))

In [189]:
class MuscleClassifier(nn.Module):
    def __init__(self, lookback):
        super().__init__()
        self.lstm = nn.LSTM(input_size=3, hidden_size=50, num_layers=2, batch_first=True)
        self.dense = nn.Linear(50, 1)
        self.inner_sigmoid = nn.Sigmoid()
        self.dense2 = nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        # print(f"input shape: {x.shape}")
        x, _ = self.lstm(x)
        # print(f"lstm output shape: {x.shape}")
        x = self.dense(x)
        # print(f"dense output shape: {x.shape}")
        # x = self.inner_sigmoid(x)
        # print(f"inner sigmoid output shape: {x.shape}")
        # x = self.dense2(x)
        # print(f"dense2 output shape: {x.shape}")
        x = self.sigmoid(x)
        # print(f"sigmoid output shape: {x.shape}")
        x = torch.flatten(x, start_dim=1)
        return x

In [190]:
model = MuscleClassifier(window)
opt = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=32, shuffle=True)

# model = model.float()

n_epoch = 50
for epoch in range(n_epoch):
    model.train()
    for x_b, y_b in loader:
        y_hat = model(x_b)
        # print(y_hat.shape, y_b.shape)
        # print(y_hat, y_b)
        loss = loss_fn(y_hat[:, -1], y_b[:, -1])
        opt.zero_grad()
        loss.backward()
        opt.step()

    # every 100 epochs, print the loss
    if epoch % (n_epoch//10) != 0:
        continue

    model.eval()
    with torch.no_grad():
        y_hat = model(X)
        loss = loss_fn(y_hat, y)
        acc = torch.abs(y_hat[:, -1] - y[:, -1]).mean()

        print(f'epoch: {epoch}, loss: {loss}, acc = {acc}')

epoch: 0, loss: 26.729875564575195, acc = 0.5020154118537903
epoch: 5, loss: 26.90545082092285, acc = 0.48402222990989685
epoch: 10, loss: 27.150135040283203, acc = 0.4739077687263489
epoch: 15, loss: 26.979591369628906, acc = 0.5240110158920288
epoch: 20, loss: 27.231870651245117, acc = 0.4729768931865692
epoch: 25, loss: 27.12268829345703, acc = 0.4714074432849884
epoch: 30, loss: 26.880788803100586, acc = 0.4727977514266968
epoch: 35, loss: 26.87194061279297, acc = 0.4727477729320526
epoch: 40, loss: 26.871925354003906, acc = 0.4727482199668884
epoch: 45, loss: 26.871929168701172, acc = 0.472748726606369
